In [1]:
# импорт необходимых библиотек
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm

import tqdm
import random

c:\VUZ\slow_fast_test\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# загрузка тренировочных данных, содержащих необходимые title и description видео
train_data_file_data = pd.read_csv("./train_data_categories.csv")
train_data_file_data

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира
...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,Массовая культура:Отношения знаменитостей: Сем...
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,Еда и напитки: Кулинария
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Спорт
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,"Книги и литература: Поэзия, Музыка и аудио, К..."


# СОСТАВЛЕНИЕ ДАТАСЕТА

In [3]:
# чтение всех возможных тегов
with open("./organized_tags.txt", "r", encoding="utf-8") as file:
    organized_tags_list = file.read().split('\n')

In [4]:
train_data_file_data = pd.read_csv("./train_data_categories.csv")
# получаем массив тегов, относящихся к конкретному видео
splitted_tag_list = [str(val).split(", ") for val in train_data_file_data['tags'].to_list()]

In [5]:
# составление строки, по которой будем производить сопоставление тегов
train_data_file_data['title_plus_description'] = train_data_file_data['title'] + " " + train_data_file_data['description']
title_plus_description_list = train_data_file_data['title_plus_description'].to_list()

In [6]:
tags_num = len(organized_tags_list)

# составление тренировочного набора данных
with open("./train_data.txt", "w", encoding="utf-8") as file:
    # читаем строку с названием и описанием видео 
    for idx, title_plus_desc in enumerate(tqdm.tqdm(title_plus_description_list)):
        # добавляем "положительные" соотношения описания и тега
        for val in splitted_tag_list[idx]:
            file.write(f"{title_plus_desc} ||| {val}".replace('\n', " ").replace("  ", " ") + f" ||| {random.randint(80, 99) / 100.0}\n")
        
        bad_examples_list_count = 0
        try_count_num = 1000
        cur_try_count = 0
        # генерируем случайные "отрицательные" соотношения описания и тега
        while (bad_examples_list_count < 3 and try_count_num > cur_try_count):
            cur_try_count += 1
            cur_tag_num = random.randint(0, tags_num - 1)
            cur_tag = organized_tags_list[cur_tag_num]
            is_skeep = False
            for val in splitted_tag_list[idx]:
                if (val in cur_tag) or (cur_tag in val):
                    is_skeep = True
                    break

            if not is_skeep:
                bad_examples_list_count += 1
                file.write(f"{title_plus_desc} ||| {cur_tag}".replace('\n', " ").replace("  ", " ") + f" ||| {random.randint(5, 20) / 100.0}\n")


                


100%|██████████| 1049/1049 [00:00<00:00, 22604.24it/s]


# КОНЕЦ СОСТАВЛЕНИЯ ДАТАСЕТА

In [8]:
import random
# создаем массив для обучения
train_data = []

# считываем тренировочные данные из файла и преобразуем их
with open("./train_data.txt", "r", encoding="utf-8") as file:
    file_data = file.read().split('\n')
    for data in file_data:
        splitted_data = data.split(" ||| ")
        try:
            train_data.append((splitted_data[0], splitted_data[1], float(splitted_data[2])))
        except:
            continue

len(train_data), train_data[1]


(4720,
 ('Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17 Тяпа и Егор бросили вызов нестареющему «повелителю ночи». Смогут ли они подарить ему статус «ЧистоТачки»?',
  'Наука: География',
  0.06))

In [9]:
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

train_examples = [InputExample(texts=[text1, text2], label=score) for text1, text2, score in train_data]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [10]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence', )

No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.


In [11]:
from sentence_transformers import losses

train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
# производим обучение модели
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10
)

In [9]:
# сохраняем обученную модель
model.save("./trained_model")

In [10]:
# проверяем успешность сохранения
model_1 = SentenceTransformer("./trained_model")

In [11]:
model_1

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)